In [6]:
from collections import deque
import itertools

In [7]:
PASSENGERS = ['red', 'blue', 'green']
BAGS = ['r1', 'r2', 'b1']

In [8]:
def unpack_state(state):
    boat = state[0]
    left_bank_passengers = set([PASSENGERS[index] for index, side in enumerate(state[1:4]) if side == 'left'])
    right_bank_passengers = set([PASSENGERS[index] for index, side in enumerate(state[1:4]) if side == 'right'])
    left_bank_bags = set([BAGS[index] for index, bag in enumerate(state[4:]) if bag == 'left'])
    right_bank_bags = set([BAGS[index] for index, bag in enumerate(state[4:]) if bag == 'right'])
    
    #print(boat, left_bank_passengers, left_bank_bags, right_bank_passengers, right_bank_bags)
    return boat, left_bank_passengers, left_bank_bags, right_bank_passengers, right_bank_bags

In [9]:
def obtain_boat_trips(available_passengers, available_bags):
    l_pass = list(available_passengers)
    l_bags = list(available_bags)
    boat_trips_passengers = [(passenger, ) for passenger in l_pass]
    boat_trips_passenger_bag = []
    if len(l_pass) >= 2:
        for possible_trip in itertools.combinations(l_pass, 2):
            boat_trips_passengers.append((possible_trip))
    for passenger in l_pass:
        for bag in l_bags:
            if passenger == 'red' and (bag == 'r1' or bag == 'r2'):
                boat_trips_passenger_bag.append((passenger, bag))
            if passenger == 'blue' and bag == 'b1':
                boat_trips_passenger_bag.append((passenger, bag))
    #print(boat_trips_passengers, boat_trips_passenger_bag)
    return boat_trips_passengers, boat_trips_passenger_bag

In [10]:
def approval(bank_passengers, bank_bags):
    if 'red' in bank_passengers and len(bank_passengers) == 1 and 'b1' in bank_bags:
        return False
    if 'blue' in bank_passengers and len(bank_passengers) == 1 and ('r1' or 'r2' in bank_bags):
        return False
    if 'green' in bank_passengers and len(bank_passengers) == 1 and bank_bags:
        return False
    return True

In [11]:
def other_side(boat):
    if boat == 'left':
        return 'right'
    else:
        return 'left'

In [12]:
def belongs(who, left_bank, right_bank):
    if who in left_bank:
        return 'left'
    elif who in right_bank:
        return 'right'
    else:
        return 'Missing!!!'

In [13]:
def expand_state(state):
    boat, left_bank_passengers, left_bank_bags, right_bank_passengers, right_bank_bags = unpack_state(state)
    new_states = []
    
    if boat == 'left':
        boat_trips_passengers, boat_trips_passenger_bag = obtain_boat_trips(left_bank_passengers, left_bank_bags)
    else:
        boat_trips_passengers, boat_trips_passenger_bag = obtain_boat_trips(right_bank_passengers, right_bank_bags)
    
    for trip_passengers in boat_trips_passengers:
        if boat == 'left':
            new_left_bank_passengers = left_bank_passengers - set(trip_passengers)
            new_right_bank_passengers = right_bank_passengers | set(trip_passengers)
            new_left_bank_bags = left_bank_bags
            new_right_bank_bags = right_bank_bags
        if boat == 'right':
            new_left_bank_passengers = left_bank_passengers | set(trip_passengers)
            new_right_bank_passengers = right_bank_passengers - set(trip_passengers)
            new_left_bank_bags = left_bank_bags
            new_right_bank_bags = right_bank_bags
            
        if approval(new_left_bank_passengers, new_left_bank_bags) and approval(new_right_bank_passengers, new_right_bank_bags):
            red = belongs('red', new_left_bank_passengers, new_right_bank_passengers)
            blue = belongs('blue', new_left_bank_passengers, new_right_bank_passengers)
            green = belongs('green', new_left_bank_passengers, new_right_bank_passengers)
            r1 = belongs('r1', new_left_bank_bags, new_right_bank_bags)
            r2 = belongs('r2', new_left_bank_bags, new_right_bank_bags)
            b1 = belongs('b1', new_left_bank_bags, new_right_bank_bags)
            new_states.append((other_side(boat), red, blue, green, r1, r2, b1))
            
    for trip_passenger_bag in boat_trips_passenger_bag:
        if boat == 'left':
            new_left_bank_passengers = left_bank_passengers - set(trip_passenger_bag[0])
            new_left_bank_bags = left_bank_bags - set(trip_passenger_bag[1])
            new_right_bank_passengers = right_bank_passengers | set(trip_passenger_bag[0])
            new_right_bank_bags = right_bank_bags | set(trip_passenger_bag[1])
        if boat == 'right':
            new_left_bank_passengers = left_bank_passengers | set(trip_passenger_bag[0])
            new_left_bank_bags = left_bank_bags | set(trip_passenger_bag[1])
            new_right_bank_passengers = right_bank_passengers - set(trip_passenger_bag[0])
            new_right_bank_bags = right_bank_bags - set(trip_passenger_bag[1]) 
            
        if approval(new_left_bank_passengers, new_left_bank_bags) and approval(new_right_bank_passengers, new_right_bank_bags):
            red = belongs('red', new_left_bank_passengers, new_right_bank_passengers)
            blue = belongs('blue', new_left_bank_passengers, new_right_bank_passengers)
            green = belongs('green', new_left_bank_passengers, new_right_bank_passengers)
            r1 = belongs('r1', new_left_bank_bags, new_right_bank_bags)
            r2 = belongs('r2', new_left_bank_bags, new_right_bank_bags)
            b1 = belongs('b1', new_left_bank_bags, new_right_bank_bags)
            new_states.append((other_side(boat), red, blue, green, r1, r2, b1))
    #print(new_left_bank_passengers, new_left_bank_bags)
    #print(left_bank_passengers)
    #print(new_states)
    return new_states

In [14]:
def search_path(initial_state, goal_state):
    visited = {initial_state}
    states_queue = deque([[initial_state]])
    while states_queue:
        states_list = states_queue.popleft()
        state_to_expand = states_list[-1]
        #print(states_list)
        for next_state in expand_state(state_to_expand):
            if next_state not in visited:
                if next_state == goal_state:
                    return states_list + [next_state]
                visited.add(next_state)
                states_queue.append(states_list + [next_state])
    #print(visited)
    #print(states_queue)
    return []

In [56]:
def visualise(path):
    if not path:
        print('Search path did not find a solution')
        return
    for pair_of_states in zip(path, path[1:]):
        boat_old, left_old, right_old = unpack_state(pair_of_states[0])
        boat_new, left_new, right_new = unpack_state(pair_of_states[1])
        delimiter_space = ' ' * 50
        separated_print(left_old)
        print(delimiter_space, end='')
        separated_print(right_old)
        print()
        if boat_old == 'left':
            delimiter = ' ' * 5 + '>' * 15 + ' ' * 5
            separated_print(left_new)
            print(delimiter, end='')
            separated_print(left_old - left_new)
            print(delimiter, end='')
            separated_print(right_old)
            print()
        else:
            delimiter = ' ' * 5 + '<' * 15 + ' ' * 5
            separated_print(left_old)
            print(delimiter, end='')
            separated_print(right_old - right_new)
            print(delimiter, end='')
            separated_print(right_new)
            print()
        separated_print(left_new)
        print(delimiter_space, end='')
        separated_print(right_new)
        print()
        print()
        print()

In [57]:
def separated_print(iterable):
    for element in iterable:
        print(element, end=' ')
    if not iterable:
        print('Empty', end='')

In [15]:
initial_state = ('left', 'left', 'left', 'left', 'left', 'left', 'left') # poslednite 3 se torbite
goal_state = ('right', 'right', 'right', 'right', 'right', 'right', 'right') # prvoto e pozicija na brod
path = search_path(initial_state, goal_state)
print(path)

[]
